In [1]:
#####################################################################################################
# PARETO was produced under the DOE Produced Water Application for Beneficial Reuse Environmental
# Impact and Treatment Optimization (PARETO), and is copyright (c) 2021-2023 by the software owners:
# The Regents of the University of California, through Lawrence Berkeley National Laboratory, et al.
# All rights reserved.
#
# NOTICE. This Software was developed under funding from the U.S. Department of Energy and the U.S.
# Government consequently retains certain rights. As such, the U.S. Government has been granted for
# itself and others acting on its behalf a paid-up, nonexclusive, irrevocable, worldwide license in
# the Software to reproduce, distribute copies to the public, prepare derivative works, and perform
# publicly and display publicly, and to permit others to do so.
#####################################################################################################

from pareto.strategic_water_management.strategic_produced_water_optimization import (
    WaterQuality,
    create_model,
    Objectives,
    solve_model,
    PipelineCost,
    PipelineCapacity,
    Hydraulics,
    RemovalEfficiencyMethod,
    InfrastructureTiming,
)
from pareto.utilities.get_data import get_data
from pareto.utilities.results import (
    plot_bars,
    plot_sankey,
    generate_report,
    PrintValues,
    OutputUnits,
    is_feasible,
    nostdout,
)
from importlib import resources

expression symbols from pyomo.core.expr  (deprecated in 6.6.2) (called from
<frozen importlib._bootstrap>:228)


In [2]:
# Each entry in set_list corresponds to a tab in the Excel input file that
# defines an index set.
set_list = [
    "ProductionPads",
    "CompletionsPads",
    "SWDSites",
    "FreshwaterSources",
    "StorageSites",
    "TreatmentSites",
    "ReuseOptions",
    "NetworkNodes",
    "PipelineDiameters",
    "StorageCapacities",
    "InjectionCapacities",
    "TreatmentCapacities",
    "TreatmentTechnologies",
]
# Each entry in parameter_list also corresponds to a tab in the Excel input
# file, but these tabs have parameter data.
parameter_list = [
    "Units",
    "PNA",
    "CNA",
    "CCA",
    "NNA",
    "NCA",
    "NKA",
    "NRA",
    "NSA",
    "FCA",
    "RCA",
    "RNA",
    "RSA",
    "SCA",
    "SNA",
    "ROA",
    "RKA",
    "SOA",
    "NOA",
    "PCT",
    "PKT",
    "FCT",
    "CST",
    "CCT",
    "CKT",
    "RST",
    "ROT",
    "SOT",
    "RKT",
    "Elevation",
    "CompletionsPadOutsideSystem",
    "DesalinationTechnologies",
    "DesalinationSites",
    "BeneficialReuseCost",
    "BeneficialReuseCredit",
    "TruckingTime",
    "CompletionsDemand",
    "PadRates",
    "FlowbackRates",
    "WellPressure",
    "NodeCapacities",
    "InitialPipelineCapacity",
    "InitialPipelineDiameters",
    "InitialDisposalCapacity",
    "InitialTreatmentCapacity",
    "ReuseMinimum",
    "ReuseCapacity",
    "FreshwaterSourcingAvailability",
    "PadOffloadingCapacity",
    "CompletionsPadStorage",
    "DisposalOperationalCost",
    "TreatmentOperationalCost",
    "ReuseOperationalCost",
    "PipelineOperationalCost",
    "FreshSourcingCost",
    "TruckingHourlyCost",
    "PipelineDiameterValues",
    "DisposalCapacityIncrements",
    "InitialStorageCapacity",
    "StorageCapacityIncrements",
    "TreatmentCapacityIncrements",
    "TreatmentEfficiency",
    "RemovalEfficiency",
    "DisposalExpansionCost",
    "StorageExpansionCost",
    "TreatmentExpansionCost",
    "PipelineCapexDistanceBased",
    "PipelineCapexCapacityBased",
    "PipelineCapacityIncrements",
    "PipelineExpansionDistance",
    "Hydraulics",
    "Economics",
    "PadWaterQuality",
    "StorageInitialWaterQuality",
    "PadStorageInitialWaterQuality",
    "DisposalOperatingCapacity",
    "TreatmentExpansionLeadTime",
    "DisposalExpansionLeadTime",
    "StorageExpansionLeadTime",
    "PipelineExpansionLeadTime_Dist",
    "PipelineExpansionLeadTime_Capac",
    "SWDDeep",
    "SWDAveragePressure",
    "SWDProxOrphanWell",
    "SWDProxInactiveWell",
    "SWDProxEQ",
    "SWDProxFault",
    "SWDProxHpOrLpWell",
    "SWDRiskFactors",
]

In [6]:
# Load data from Excel input file into Python
with resources.path(
    "pareto.case_studies",
    "strategic_toy_case_risk_study.xlsx",
) as fpath:
    [df_sets, df_parameters] = get_data(fpath, set_list, parameter_list)

In [14]:
def calculate_site_risk(df_sets, df_parameters):
    sites = df_sets["SWDSites"]
    deep = df_parameters["SWDDeep"]
    pressure = df_parameters["SWDAveragePressure"]
    prox = {
        "orphan": df_parameters["SWDProxOrphanWell"],
        "inactive": df_parameters["SWDProxInactiveWell"],
        "eq": df_parameters["SWDProxEQ"],
        "fault": df_parameters["SWDProxFault"],
        "hplp": df_parameters["SWDProxHpOrLpWell"],
    }
    risk_factors = df_parameters["SWDRiskFactors"]

    risk_factor_names = {
        "orphan": "orphan_well",
        "inactive": "inactive_well",
        "eq": "EQ",
        "fault": "fault",
        "hplp": "HP_LP",
    }

    distance_risk_factors = {}
    severity_risk_factors = {}
    sum_risk_factors = {
        "distance": 0,
        "severity": 0,
    }
    for factor, name in risk_factor_names.items():
        distance_risk_factor = distance_risk_factors[factor] = risk_factors[f"{name}_distance_risk_factor"]
        severity_risk_factor = severity_risk_factors[factor] = risk_factors[f"{name}_severity_risk_factor"]
        sum_risk_factors["distance"] += distance_risk_factor
        sum_risk_factors["severity"] += severity_risk_factor
        
    for factor in risk_factor_names:
        distance_risk_factors[f"norm_{factor}"] = distance_risk_factors[factor] / sum_risk_factors["distance"]
        severity_risk_factors[f"norm_{factor}"] = severity_risk_factors[factor] / sum_risk_factors["severity"]

    pressure_thresholds = {
        "max": risk_factors["HP_threshold"],
        "min": risk_factors["LP_threshold"],
    }

    max_risk_factor_for_lowest_risk = 0
    for factor in risk_factor_names:
        norm_factor = f"norm_{factor}"
        max_risk_factor_for_lowest_risk += distance_risk_factors[factor] * distance_risk_factors[norm_factor] * severity_risk_factors[norm_factor]

    site_risk_metrics = {}
    site_included = {}
    for site in sites:
        weighted_site_risk_factor = 0
        for factor in risk_factor_names:
            if factor in ("orphan", "inactive") and deep[site]:
                site_risk_factor = distance_risk_factors[factor]
            else:
                site_risk_factor = min(prox[factor][site], distance_risk_factors[factor])
            norm_factor = f"norm_{factor}"
            weighted_site_risk_factor += site_risk_factor * distance_risk_factors[norm_factor] * severity_risk_factors[norm_factor]
        inverse_weighted_site_risk_factor = weighted_site_risk_factor / max_risk_factor_for_lowest_risk
        site_risk_metrics[site] = 1 - inverse_weighted_site_risk_factor
        # only consider these sites with True site_included!
        site_included[site] = pressure_thresholds["min"] <= pressure[site] < pressure_thresholds["max"]

    return site_risk_metrics, site_included

site_risk_metrics, site_included = calculate_site_risk(df_sets, df_parameters)

# from Laura
test_site_risk_metrics = {
    "K06": 0.25,
    "K07": 0.11,
    "K08": 0.17,
    "K09": 0.12,
    "K10": 0.08,
    "K11": 0.26,
    "K12": 0.28,
    "K13": 0.09,
    "K14": 0.20,
}
test_site_included = {
    "K06": False,
    "K07": True,
    "K08": True,
    "K09": True,
    "K10": True,
    "K11": True,
    "K12": True,
    "K13": True,
    "K14": True,
}

passed = True
for site in df_sets["SWDSites"]:
    if abs(site_risk_metrics[site] - test_site_risk_metrics[site]) > 0.01 or site_included[site] != test_site_included[site]:
        passed = False
        break
if passed:
    print("Test passed")
else:
    print("Test failed")

Test passed
